In [1]:
import numpy as np 
import pandas as pd
from astropy.io import fits
import sys
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

A new version of galpy (1.9.2) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [ ]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
#dfv = dfv.loc[dfv['extinction'].between(0.0003, 1)]
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['X'].values[::ds_index],
    y=dfv['Y'].values[::ds_index],
    z=dfv['Z'].values[::ds_index],
    mode='markers',
    marker=dict(size=5.,
                color='gray',
                # color=dfv['extinction'].values[::ds_index],
                # colorscale='Gray_r',
                cmin = 0.0007,
                cmax = 0.005,
                symbol='circle',
                opacity=.01),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0


hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

In [ ]:
# RW/Split/CS clusters
rw_region = pd.read_csv('/Users/cam/Downloads/rw_clusters.csv')
rw = rw_region.loc[rw_region.age_myr < 15]
remove = ['Theia_6', 'Teutsch_39', 'OCSN_32', 'Aveni-Hunter_1', 'IC_5146', 'Collinder_419', 'Dolidze_8', 'Theia_1920', 'Collinder_95', 'Mon_OB1-D', 'NGC_2264']
rw = rw[~rw['name'].isin(remove)]
rw = rw.rename(columns = column_renaming_dict)
#lacerta = ['HSC_661', 'vdBergh_152', 'HSC_705', 'OCSN_32', 'UBC_159', 'UPK_168', 'UPK_166', 'Teutch_39', 'Theia_100']

# The Split
split_region = pd.read_csv('/Users/cam/Downloads/the_split_selection.csv')
split = split_region.loc[split_region.age_myr < 15]
split = split.rename(columns = column_renaming_dict)

# Cepheus spur
cepheus_region = pd.read_csv('/Users/cam/Downloads/cepheus_spur_region.csv')
cepheus_spur = cepheus_region.loc[cepheus_region.age_myr < 30]
cepheus_spur = cepheus_spur.rename(columns = column_renaming_dict)

In [ ]:
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

df_hunt_good = df_hunt_full.loc[
    (df_hunt_full['n_rv_gaia_apogee_galah'] >= 2) & 
    (df_hunt_full['U_err'] < 10) & 
    (df_hunt_full['V_err'] < 10) & 
    (df_hunt_full['W_err'] < 10)
    ]
df_hunt_young = df_hunt_good.loc[df_hunt_good['age_myr'] < 15]
df_hunt_middle_aged = df_hunt_good.loc[df_hunt_good['age_myr'].between(15, 60)]
df_hunt_older = df_hunt_good.loc[df_hunt_good['age_myr'].between(0, 60)]
df_hunt_older.to_csv('/Users/cam/Downloads/hunt_catalog_0_30_myr.csv', index=False)

In [ ]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [0.], 'n_stars' : [1]})


In [ ]:
rw_data = StarClusterData(rw, data_name = 'Radcliffe Wave', min_size = 0, max_size = 9, color = 'red', opacity = 1., marker_style = 'circle')
split_data = StarClusterData(split, data_name = 'The Split', min_size = 0, max_size = 9, color = '#FFA07A', opacity = 1., marker_style = 'circle')
cepheus_spur_data = StarClusterData(cepheus_spur, data_name = 'Cepheus Spur', min_size = 0, max_size = 9, color = 'cyan', opacity = 1., marker_style = 'circle')

ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 7, color = 'violet', opacity = 1., marker_style = 'circle')
m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 7, color = 'cyan', opacity = 1., marker_style = 'circle')
cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 7, color = 'orange', opacity = 1., marker_style = 'circle')
gamma_vel_data = StarClusterData(gamma_vel, data_name = '𝝲Vel', min_size = 0, max_size = 7, color = 'red', opacity = 1., marker_style = 'circle')
young_data = StarClusterData(df_hunt_young, data_name = 'Young clusters', min_size = 2., max_size = 7, color = 'black', opacity = 1., marker_style = 'circle')
older_data = StarClusterData(df_hunt_older, data_name = 'Older clusters', min_size = 0., max_size = 7, color = 'black', opacity = 1., marker_style = 'circle')
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 7, color = 'yellow', opacity = 1., marker_style = 'circle')


sc = StarClusterCollection([young_data, older_data, sun_data, ap_data, m6_data, cr135_data, gamma_vel_data, rw_data, split_data, cepheus_spur_data])
grouping_dict = {
    'Cluster families' : ['⍺Per', 'M6', 'Cr135', '𝝲Vel', 'Sun'],
    'Dust spurs/spines' : ['Young clusters', 'Radcliffe Wave', 'The Split', 'Cepheus Spur', 'Sun'],
    'Older clusters' : ['Older clusters', 'Sun']
}